In [ ]:
pip install langchain transformers

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline

In [4]:
hardcoded_text = """

# EDS UI Library Documentation

## Buttons
- Use `eds-btn` for buttons.
- Use `eds-btn-primary` for primary buttons.
- Use `eds-btn-secondary` for secondary buttons.

Example:
<button class="eds-btn eds-btn-primary">Primary Button</button>
<button class="eds-btn eds-btn-secondary">Secondary Button</button>

## Containers
- Use `eds-container` for a fixed-width container.
- Use `eds-container-fluid` for a full-width container.

Example:
<div class="eds-container">
  Fixed-width content goes here.
</div>
<div class="eds-container-fluid">
  Full-width content goes here.
</div>

## Forms
- Use `eds-form` for creating forms.
- Use `eds-input` for form inputs.

Example:
<div class="eds-form">
  <label for="username">Username</label>
  <input type="text" id="username" class="eds-input" placeholder="Enter your username">
  
  <label for="password">Password</label>
  <input type="password" id="password" class="eds-input" placeholder="Enter your password">
  
  <button class="eds-btn eds-btn-primary">Login</button>
</div>

## Inputs
- Use `eds-input` for text inputs.
- Use `eds-input-lg` for large inputs.
- Use `eds-input-sm` for small inputs.
- Use `eds-input-disabled` for disabled inputs.
- Use `eds-textarea` for multiline inputs.

Example:
<input type="text" class="eds-input" placeholder="Enter text">
<input type="text" class="eds-input eds-input-lg" placeholder="Large input">
<textarea class="eds-textarea" placeholder="Enter multiline text"></textarea>
<input type="text" class="eds-input eds-input-sm" placeholder="Small input">
<input type="text" class="eds-input eds-input-disabled" placeholder="Disabled input" disabled>

## Data Tables
- Use `eds-table` for basic tables.
- Use `eds-table-striped` for striped rows.
- Use `eds-table-bordered` for tables with borders.
- Use `eds-table-hover` for hoverable rows.
- Use `eds-table-sm` for smaller table sizes.

Example:
<table class="eds-table eds-table-striped eds-table-bordered">
  <thead>
    <tr>
      <th>Name</th>
      <th>Age</th>
      <th>Country</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>John Doe</td>
      <td>28</td>
      <td>USA</td>
    </tr>
    <tr>
      <td>Jane Smith</td>
      <td>32</td>
      <td>Canada</td>
    </tr>
  </tbody>
</table>

## Cards
- Use `eds-card` for basic card.
- Use `eds-card-header` for card headers.
- Use `eds-card-body` for the body of the card.
- Use `eds-card-footer` for card footers.

Example:
<div class="eds-card">
  <div class="eds-card-header">
    Card Header
  </div>
  <div class="eds-card-body">
    This is the body of the card.
  </div>
  <div class="eds-card-footer">
    Card Footer
  </div>
</div>

## Modals
- Use `eds-modal` for modals.
- Use `eds-modal-header` for the modal header.
- Use `eds-modal-body` for the modal body.
- Use `eds-modal-footer` for the modal footer.

Example:
<div class="eds-modal">
  <div class="eds-modal-header">
    Modal Header
  </div>
  <div class="eds-modal-body">
    This is the modal body.
  </div>
  <div class="eds-modal-footer">
    <button class="eds-btn eds-btn-secondary">Close</button>
  </div>
</div>

## Navigation
- Use `eds-nav` for navigation bars.
- Use `eds-nav-item` for each navigation item.
- Use `eds-nav-link` for the navigation links.

Example:
<nav class="eds-nav">
  <a class="eds-nav-link eds-nav-item" href="#">Home</a>
  <a class="eds-nav-link eds-nav-item" href="#">About</a>
  <a class="eds-nav-link eds-nav-item" href="#">Services</a>
  <a class="eds-nav-link eds-nav-item" href="#">Contact</a>
</nav>

## Alerts
- Use `eds-alert` for alerts.
- Use `eds-alert-success` for success alerts.
- Use `eds-alert-info` for informational alerts.
- Use `eds-alert-warning` for warning alerts.
- Use `eds-alert-danger` for danger alerts.

Example:
<div class="eds-alert eds-alert-success">
  This is a success alert!
</div>
<div class="eds-alert eds-alert-danger">
  This is a danger alert!
</div>

## Pagination
- Use `eds-pagination` for basic pagination.
- Use `eds-pagination-item` for pagination items.
- Use `eds-pagination-link` for pagination links.

Example:
<ul class="eds-pagination">
  <li class="eds-pagination-item">
    <a href="#" class="eds-pagination-link">&laquo;</a>
  </li>
  <li class="eds-pagination-item">
    <a href="#" class="eds-pagination-link">1</a>
  </li>
  <li class="eds-pagination-item">
    <a href="#" class="eds-pagination-link">2</a>
  </li>
  <li class="eds-pagination-item">
    <a href="#" class="eds-pagination-link">&raquo;</a>
  </li>
</ul>

"""

In [5]:
from langchain.schema import Document

In [6]:
documents = [{"page_content": hardcoded_text, "metadata": {"key": "value"}}]

# Convert to LangChain Document objects
docs = [Document(page_content=doc["page_content"], metadata=doc.get("metadata", {})) for doc in documents]

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectordb = FAISS.from_documents(docs, embedding_model)

In [ ]:
llm_pipeline = pipeline(
    "text-generation",
    model="EleutherAI/gpt-neo-1.3B",
    device=-1,
    pad_token_id=50256,
    max_new_tokens=200,  # Reserve 200 tokens for output
    max_length=700,  # Ensure total token length (input + output) doesn't exceed 700
)



In [15]:
llm = HuggingFacePipeline(pipeline=llm_pipeline)

In [16]:
template = """
Use the following pieces of context to answer the question at the end.

If the question asks for a button, input, table, or any HTML code, provide the relevant information in plain text. Clearly explain how to use the component in EDS or standard HTML with an example.

If the input contains HTML code, identify the EDS-specific equivalent or alternative for the HTML. Provide the answer with examples in EDS-specific syntax.

If the question is not related to components, answer it in plain text using the context provided. 
If you don't know the answer, just say that you don't know and avoid making up an answer.

Use three sentences maximum. Always say "Thanks for asking!" at the end.

Context:
{context}

Question: {question}

Helpful Answer:
"""

In [11]:
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:

result = qa_chain({"query": 'button'})

In [2]:

pip install jq


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import streamlit as st
import  jq
# Define the schema to load JSON
jq_schema = {
    "content": ".content"  # Assuming the content of the document is under the "content" key in the JSON
}

# Load JSON file (replace with your actual JSON file path)
file_path = "eds_data.json"  # Replace with your file path
loader = JSONLoader(file_path, jq_schema=jq_schema)
documents = loader.load()

# Ensure that the documents are in the correct format (only extract 'content' as text)
extracted_docs = [doc['content'] for doc in documents]  # Only extract 'content' field from each document

# Check if we have correctly extracted text
print(extracted_docs) 
